# Backpropagation handmade

In [31]:
# 1. Build the class for math expressions
# 2. Build some basic operations
# 3. Build computation graph
# 4. Build some activations
# 5. Build backpropagation

In [32]:
# inspired by (and 80% copied from :)) micrograd / Andrej Karpathy
# https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ

In [33]:
import numpy as np

import math
import matplotlib.pyplot as plt
from collections import defaultdict, deque

In [47]:
class Parameter:
    def __init__(self, value: float, name: str) -> None:
        self._value = value
        self._name = name
        self._grad = 0.0

        self._graph = defaultdict(list)
        self._graph[id(self)] = list()

        self._params = defaultdict()
        self._params[id(self)] = self

        self._backward = lambda: None
    def __repr__(self) -> str:
        return f"Parameter {self._name} = {self._value}; dL/d[{self._name}] = {self._grad}"

    def __mul__(self, other):
        result = Parameter(
            self._value * other._value,
            f'{self._name} * {other._name}'
        )
        def _backward():
            self._grad += other._value * result._grad #dL / dself
            other._grad += self._value * result._grad # dL / dother

        result._backward = _backward

        result._params.update(self._params)
        result._params.update(other._params)
        result._params[id(result)] = result

        result._graph.update(self._graph)
        result._graph.update(other._graph)
        result._graph[id(result)].append(id(self))
        result._graph[id(result)].append(id(other))

        return result

    def __add__(self, other) :
        result = Parameter(
            self._value + other._value,
            f'[{self._name} + {other._name}]'
        )

        def _backward():
            self._grad += 1.0 * result._grad  #dL / dself
            other._grad += 1.0 * result._grad # dL / dother

        result._backward = _backward
        
        result._params.update(self._params)
        result._params.update(other._params)
        result._params[id(result)] = result

        result._graph.update(self._graph)
        result._graph.update(other._graph)
        result._graph[id(result)].append(id(self))
        result._graph[id(result)].append(id(other))


        return result

    
    def _topological_sort(self):
        in_degree = {node: 0 for node in self._graph}
        for node in self._graph:
            for child in self._graph[node]:
                in_degree[child] += 1

        queue = deque([node for node in self._graph if in_degree[node] == 0])
        sorted_nodes = []

        while queue:
            node = queue.popleft()
            sorted_nodes.append(node)
            for child in self._graph[node]:
                in_degree[child] -= 1
                if in_degree[child] == 0:
                    queue.append(child)

        return sorted_nodes

    def backward(self):
        queue = self._topological_sort()
        self._grad = 1
        for i in queue:
            self._params[i]._backward()
        

    def sigmoid(self) :
        # f(x) = 1 / (1 + exp(self._value))
        # f'(x) = f(x) * (1 - f(x))

        val = 1.0 / (1.0 + math.exp(-self._value))

        result = Parameter(
            val,
            f"σ({self._name})"
        )

        def _backward():
            self._grad = result._grad * val * (1 - val)

        result._backward = _backward

        return result

    def ReLU(self):
        val = np.maximum(0,self._value)
        result = Parameter(
            val,
            f"σ({self._name})"
        )

        result._params.update(self._params)
        result._params[id(result)] = result
        
        result._graph.update(self._graph)
        result._graph[id(result)].append(id(self))

        def _backward():
            self._grad = result._grad * (val > 0)

        result._backward = _backward

        return result

    def SoftPlus(self):
        #f(x) = ln(1+e^x)
        #f'(x) = e^x/(1+e^x) =sigmoid(x)
        val = np.log(1+np.exp(self._value))

        result = Parameter(
            val,
            f"σ({self._name})"
        )

        result._params.update(self._params)
        result._params[id(result)] = result
        
        result._graph.update(self._graph)
        result._graph[id(result)].append(id(self))

        def _backward():
            self._grad = result._grad * (1.0 / (1.0 + math.exp(-self._value)))

        result._backward = _backward

        return result

def sgd(parameters: list, learning_rate = 0.3): 
    for param in parameters:
        param._value -= learning_rate * param._grad        
    return parameters

In [53]:
a = Parameter(3.0, 'a')
b = Parameter(2.0, 'b')
c = Parameter(5.0, 'c')
d = Parameter(5.0, 'd')
a,b,c,d

(Parameter a = 3.0; dL/d[a] = 0.0,
 Parameter b = 2.0; dL/d[b] = 0.0,
 Parameter c = 5.0; dL/d[c] = 0.0,
 Parameter d = 5.0; dL/d[d] = 0.0)

In [54]:
u = a * b
v = u + c
L = v * d

L.backward()
u,v,L

(Parameter a * b = 6.0; dL/d[a * b] = 5.0,
 Parameter [a * b + c] = 11.0; dL/d[[a * b + c]] = 5.0,
 Parameter [a * b + c] * d = 55.0; dL/d[[a * b + c] * d] = 1)

In [55]:
x = Parameter(4.0, 'x')
y = Parameter(5.0, 'y')
w = Parameter(3.0, 'w')

u1 = x + y
u2 = u1 * x
u3 = u2 + w
L = u1 + u3
L.backward()
x,y,L

(Parameter x = 4.0; dL/d[x] = 14.0,
 Parameter y = 5.0; dL/d[y] = 5.0,
 Parameter [[x + y] + [[x + y] * x + w]] = 48.0; dL/d[[[x + y] + [[x + y] * x + w]]] = 1)

In [62]:
x = Parameter(3,"x")
y = Parameter(5,"y")
L = y*x*x*y*x*y
L.ReLU()
L.backward()
x,y,L

(Parameter x = 3; dL/d[x] = 3375.0,
 Parameter y = 5; dL/d[y] = 2025.0,
 Parameter y * x * x * y * x * y = 3375; dL/d[y * x * x * y * x * y] = 1)

In [64]:
import torch
x = torch.tensor(3.0, requires_grad=True)
y = torch.tensor(5.0, requires_grad=True)

L = y*x*x*y*x*y
L.relu()
L.backward()
x.grad,y.grad

(tensor(3375.), tensor(2025.), tensor(3375., grad_fn=<MulBackward0>))